In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

In [5]:
batch_size = 128
epochs = 10
lr = 1e-3
mc_samples = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(0)

In [6]:
class CNN(nn.Module):
    def __init__(self, dropout_p=0.3, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=dropout_p)

        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)

        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)

        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [7]:
transform_cifar = transforms.Compose([
    transforms.ToTensor()
])

transform_mnist = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.expand(3, -1, -1))
])

train_id = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
test_id = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)
test_ood = datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)

train_id_loader = DataLoader(train_id, batch_size=batch_size, shuffle=True)
test_id_loader = DataLoader(test_id, batch_size=batch_size, shuffle=False)
test_ood_loader = DataLoader(test_ood, batch_size=batch_size, shuffle=False)

100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [02:44<00:00, 1.04MB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:13<00:00, 717kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 212kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:02<00:00, 696kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 2.25MB/s]


In [8]:
def train(model, train_loader, epochs, lr):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        correct_samples = 0

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()

            logits = model(x)
            loss = loss_function(logits, y)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = logits.argmax(dim=1)

            correct_samples += (preds == y).sum().item()

        print(f'Epoch {epoch}; Train loss {total_loss / len(train_loader)}; Accuracy {correct_samples / len(train_loader) * 100}')

In [24]:
def compute_ood_metrics(id_scores, ood_scores):
    y_true = np.concatenate([
        np.zeros_like(id_scores),
        np.ones_like(ood_scores)
    ])
    scores = np.concatenate([id_scores, ood_scores])

    auroc = roc_auc_score(y_true, scores)
    aupr = average_precision_score(y_true, scores)

    fpr, tpr, _ = roc_curve(y_true, scores)
    target_tpr = 0.95
    idxs = np.where(tpr >= target_tpr)[0]
    if len(idxs) > 0:
        fpr95 = fpr[idxs[0]]
    else:
        fpr95 = 1.0

    print(f'AUROC {auroc}')
    print(f'AUPR {aupr}')
    print(f'FPR@95%TPR {fpr95}')

    return auroc, aupr, fpr95

In [9]:
def get_softmax_ood_scores(model, id_loader, ood_loader):
    model.to(device)
    model.eval()

    id_scores = []
    ood_scores = []

    with torch.no_grad():
        for x, _ in id_loader:
            x = x.to(device)
            logits = model(x)
            probs = F.softmax(logits, dim=1)
            max_probs, _ = probs.max(dim=1)
            scores = 1.0 - max_probs

            id_scores.append(scores.cpu().numpy())

    with torch.no_grad():
        for x, _ in ood_loader:
            x = x.to(device)
            logits = model(x)
            probs = F.softmax(logits, dim=1)
            max_probs, _ = probs.max(dim=1)
            scores = 1.0 - max_probs

            ood_scores.append(scores.cpu().numpy())

    id_scores = np.concatenate(id_scores)
    ood_scores = np.concatenate(ood_scores)

    return id_scores, ood_scores

In [10]:
def get_mcd_ood_entropy(model, x, T=20):
    model.to(device)
    model.train()

    with torch.no_grad():
        probs_T = []
        for _ in range(T):
            logits = model(x)
            probs = F.softmax(logits, dim=1)

            probs_T.append(probs.unsqueeze(0))

        probs_T = torch.cat(probs_T, dim=0)

    p_mean = probs_T.mean(dim=0)

    eps = 1e-8
    entropy = -torch.sum(p_mean * torch.log(p_mean + eps), dim=1)

    return entropy


def get_mcd_ood_scores(model, id_loader, ood_loader, T=20):
    model.to(device)

    id_scores = []
    ood_scores = []

    for x, _ in id_loader:
        x = x.to(device)
        entropy = get_mcd_ood_entropy(model, x, T=T)

        id_scores.append(entropy.cpu().numpy())

    for x, _ in ood_loader:
        x = x.to(device)
        entropy = get_mcd_ood_entropy(model, x, T=T)

        ood_scores.append(entropy.cpu().numpy())

    id_scores = np.concatenate(id_scores)
    ood_scores = np.concatenate(ood_scores)

    return id_scores, ood_scores

In [33]:
def forward_react(model, x):
    x = model.pool(F.relu(model.conv1(x)))
    x = model.dropout(x)

    x = model.pool(F.relu(model.conv2(x)))
    x = model.dropout(x)

    x = model.pool(F.relu(model.conv3(x)))
    x = model.dropout(x)

    x = x.view(x.size(0), -1)
    x = F.relu(model.fc1(x))
    x = model.dropout(x)
    # x = self.fc2(x) <- no need

    return x

def threshold_react(model, id_loader, p):
    model.to(device)
    model.eval()

    act = []

    with torch.no_grad():
        for x, _ in id_loader:
            x = x.to(device)
            h = forward_react(model, x)
            act.append(h.cpu().numpy().reshape(-1))
    act = np.concatenate(act)
    c = np.percentile(act, p)

    return c
    
def logits_react(model, x, c):
    h = forward_react(model, x)
    h_ = torch.clamp(h, max=c)
    logits = model.fc2(h_)

    return logits
    
def get_react_scores(model, id_loader, ood_loader, c):
    model.to(device)
    model.eval()

    id_scores = []
    ood_scores = []
    
    with torch.no_grad():
        for x, _ in tqdm(id_loader):
            x = x.to(device)
            logits = logits_react(model, x, c)
            probs = F.softmax(logits, dim=1)
            max_probs, _ = probs.max(dim=1)
            scores = 1.0 - max_probs

            id_scores.append(scores.cpu().numpy())

        for x, _ in tqdm(ood_loader):
            x = x.to(device)
            logits = logits_react(model, x, c)
            probs = F.softmax(logits, dim=1)
            max_probs, _ = probs.max(dim=1)
            scores = 1.0 - max_probs

            ood_scores.append(scores.cpu().numpy())

    id_scores = np.concatenate(id_scores)
    ood_scores = np.concatenate(ood_scores)
    return id_scores, ood_scores

In [21]:
model = CNN(dropout_p=0.3, num_classes=10)
train(model, train_id_loader, epochs=epochs, lr=lr)

Epoch 1; Train loss 1.7089863399715375; Accuracy 4749.104859335039
Epoch 2; Train loss 1.3422248171418525; Accuracy 6557.033248081841
Epoch 3; Train loss 1.2008957129610165; Accuracy 7304.60358056266
Epoch 4; Train loss 1.1038484295920643; Accuracy 7738.6189258312015
Epoch 5; Train loss 1.0276720807375506; Accuracy 8124.296675191816
Epoch 6; Train loss 0.981457080072759; Accuracy 8340.153452685423
Epoch 7; Train loss 0.9372576240383451; Accuracy 8564.961636828644
Epoch 8; Train loss 0.8913818367606844; Accuracy 8771.355498721228
Epoch 9; Train loss 0.8651689771191239; Accuracy 8890.281329923273
Epoch 10; Train loss 0.8346030863044817; Accuracy 9008.43989769821


In [22]:
softmax_id_scores, softmax_ood_scores = get_softmax_ood_scores(model, test_id_loader, test_ood_loader)

In [25]:
auroc, aupr, fpr95 = compute_ood_metrics(softmax_id_scores, softmax_ood_scores)

AUROC 0.6337214549999999
AUPR 0.5655969004223869
FPR@95%TPR 0.7362


In [26]:
mcd_id_scores, mcd_ood_scores = get_mcd_ood_scores(model, test_id_loader, test_ood_loader)

In [27]:
mcd_auroc, mcd_aupr, mcd_fpr95 = compute_ood_metrics(mcd_id_scores, mcd_ood_scores)

AUROC 0.71740361
AUPR 0.6185196225680331
FPR@95%TPR 0.5938


In [34]:
c_ = threshold_react(model, train_id_loader, p=90)
react_id_scores, react_ood_scores = get_react_scores(model, test_id_loader, test_ood_loader, c_)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:14<00:00,  5.58it/s]


In [36]:
react_auroc, react_aupr, react_fpr95 = compute_ood_metrics(react_id_scores, react_ood_scores)

AUROC 0.8661906950000001
AUPR 0.8331711571360616
FPR@95%TPR 0.3932


**Вывод:** Сравнивая результаты MCD и Softmax, мы можем заметить, что ReAct (в сочетании с клиппингом) дает положительный рост точности классификации в значениях на ID выборке.